In [ ]:
from paus_utils import *
from MyMocks.Make_LAE import synth_phot
import pickle
from paus_utils import w_central

In [ ]:
def gaussian(w_Arr, mean, std):
    return np.exp(-(w_Arr - mean) ** 2 / (2 * std ** 2))

In [ ]:
# Load tcurves
path_to_paus_data = '/home/alberto/almacen/PAUS_data'
with open(f'{path_to_paus_data}/paus_tcurves.pkl', 'rb') as f:
    tcurves = pickle.load(f)

In [ ]:
# Generate random flat spectra with an emission line
N_spec = 10_000
w_lya = 1215.67

f_cont = 1e-17
ew_line_rand = np.random.uniform(10, 200, N_spec)
z_lya_rand = np.random.uniform(2.6, 4.5, N_spec)
w_lya_obs_Arr = w_lya * (1 + z_lya_rand)

w_Arr = np.arange(3000, 10000, 1)

SEDs_mat = np.empty((N_spec, 40))

for src in range(N_spec):
    print(src)
    w_lya_obs = w_lya_obs_Arr[src]
    emline = gaussian(w_Arr, w_lya_obs, 100)
    # normalize the emline
    emline = emline / emline.sum() * f_cont * ew_line_rand[src]

    this_spec = np.ones(len(w_Arr)) * f_cont + emline
    
    this_SED = synth_phot(this_spec, w_Arr, tcurves)[:40]

    SEDs_mat[src] = this_SED

In [ ]:
z_pred = w_central[np.argmax(SEDs_mat, axis=1)] / w_lya - 1

In [ ]:
fig, ax = plt.subplots()

ax.scatter(z_lya_rand, z_pred)

ax.plot([2.7, 4.5], [2.7, 4.5], c='orange')

plt.show()